In [35]:
import pandas as pd
import numpy as np

In [36]:
# 读取C:\MyProject\MIMIC-IPE\data\9568例未完成的下一次住院时间计算表.csv
df = pd.read_csv(r'C:\MyProject\MIMIC-IPE\data\9568例未完成的下一次住院时间计算表.csv')

In [37]:
import pandas as pd

# 假设df是你的数据框
# 转换日期时间列
df['admittime'] = pd.to_datetime(df['admittime'])
df['dischtime'] = pd.to_datetime(df['dischtime'])

# 确保数据按照subject_id和admittime排序
df.sort_values(['subject_id', 'admittime'], inplace=True)

# 标记next_PE为1的行
df['pe_mark'] = df['next_PE'].eq(1)

# 对每个subject_id计算时间差
def calculate_time_diff(group):
    # 选取next_PE为1的行的索引
    pe_indices = group.index[group['pe_mark']]
    if len(pe_indices) > 1:
        # 取第一个和第二个next_PE为1的时间
        first_pe_time = group.loc[pe_indices[0], 'dischtime']
        second_pe_time = group.loc[pe_indices[1], 'admittime']
        # 计算时间差并转换为天数
        time_diff = (second_pe_time - first_pe_time).days
        group.loc[pe_indices[1], 'next_PE_day'] = time_diff
    return group

# 应用函数到每个subject_id组
df = df.groupby('subject_id').apply(calculate_time_diff)

# 清理辅助列
# df.drop('pe_mark', axis=1, inplace=True)


C:\Users\ZhouNan\AppData\Local\Temp\ipykernel_9732\477191911.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('subject_id').apply(calculate_time_diff)


In [38]:
# 重置索引
df.reset_index(drop=True, inplace=True)

In [39]:
import pandas as pd
import numpy as np

def process_group(group):
    # 保留pe_mark为True的行的hadm_id值
    first_hadm_id = group.loc[group['pe_mark'], 'hadm_id'].values[0]

    # 检查是否存在next_PE_day非NaN的行
    if group['next_PE_day'].notna().any():
        # 优先保留next_PE_day非NaN所在行的数据
        group = group[group['next_PE_day'].notna()]
    else:
        # 如果next_PE_day全部为NaN，则将next_PE设为0
        group['next_PE'] = 0

    # hadm_id列使用pe_mark为True的行的hadm_id值
    group['hadm_id'] = first_hadm_id

    # 保留每列的第一个非NaN值
    return group.ffill().bfill().iloc[0]

# 应用自定义函数处理每个分组
result_df = df.groupby('subject_id').apply(process_group).reset_index(drop=True)

print(result_df)


      subject_id   hadm_id           admittime           dischtime  seq_hosp  \
0       10001401  24818636 2131-06-04 00:00:00 2131-06-15 16:10:00         1   
1       10002221  21008195 2200-09-29 17:28:00 2200-10-01 14:58:00         1   
2       10015272  27993466 2137-06-12 18:36:00 2137-06-18 15:45:00         1   
3       10015568  26581506 2143-08-19 18:11:00 2143-08-22 18:54:00         1   
4       10018928  22523751 2125-02-27 08:58:00 2125-03-10 17:00:00         1   
...          ...       ...                 ...                 ...       ...   
2034    19990141  24852269 2133-03-04 17:34:00 2133-03-05 17:30:00         1   
2035    19991135  29872770 2124-02-17 08:30:00 2124-02-20 08:50:00         1   
2036    19995127  27369164 2138-03-07 14:04:00 2138-03-11 14:53:00         1   
2037    19995320  21633275 2136-10-05 00:31:00 2136-10-06 21:11:00         1   
2038    19997576  25548363 2187-10-07 22:35:00 2187-10-10 19:50:00         1   

      next_PE  pe_mark  next_PE_day  
0

C:\Users\ZhouNan\AppData\Local\Temp\ipykernel_9732\27726977.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = df.groupby('subject_id').apply(process_group).reset_index(drop=True)


In [40]:
# 读取C:\MyProject\MIMIC-IPE\加了中介变量的数据集\再入院情况表.csv
df2 = pd.read_csv(r'C:\MyProject\MIMIC-IPE\加了中介变量的数据集\再入院情况表.csv')

In [41]:
# 使用 'next_PE_day' 列的最大值填充 NaN 值
result_df['next_PE_day'].fillna(result_df['next_PE_day'].max(), inplace=True)

# 合并 df 和 df2，保留df2的所有行和所有列
merged_df = pd.merge(result_df[['subject_id', 'hadm_id', 'seq_hosp', 'next_PE', 'next_PE_day']], df2, on=['subject_id', 'hadm_id'], how='right')
merged_df

C:\Users\ZhouNan\AppData\Local\Temp\ipykernel_9732\2097299534.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result_df['next_PE_day'].fillna(result_df['next_PE_day'].max(), inplace=True)


,subject_id,hadm_id,seq_hosp,next_PE,next_PE_day,Group_Label,re_inhosp,re_inhosp_time,next_admission_type,first_hosp
0,10001401,24818636,1,0,3206.0,DVT-PE,0,NaN,OBSERVATION ADMIT,0
1,10002221,21008195,1,0,3206.0,IPE,0,NaN,EW EMER.,1
2,10015272,27993466,1,0,3206.0,DVT-PE,0,NaN,EW EMER.,1
3,10015568,26581506,1,0,3206.0,IPE,0,NaN,EW EMER.,1
4,10018928,22523751,1,0,3206.0,IPE,0,NaN,EW EMER.,1
...,...,...,...,...,...,...,...,...,...,...
2034,19990141,24852269,1,0,3206.0,DVT-PE,0,NaN,OBSERVATION ADMIT,1
2035,19991135,29872770,1,0,3206.0,DVT-PE,0,NaN,EW EMER.,0
2036,19995127,27369164,1,0,3206.0,IPE,0,NaN,EW EMER.,0
2037,19995320,21633275,1,0,3206.0,IPE,0,NaN,EW EMER.,1


In [42]:
# next_PE列缺失值使用0值填充
merged_df['next_PE'].fillna(0, inplace=True)


C:\Users\ZhouNan\AppData\Local\Temp\ipykernel_9732\2269056706.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['next_PE'].fillna(0, inplace=True)


In [43]:
# merged_df保存到C:\MyProject\MIMIC-IPE\加了中介变量的数据集\下一次PE入院情况表.csv
merged_df.to_csv(r'C:\MyProject\MIMIC-IPE\加了中介变量的数据集\下一次PE入院情况表.csv', index=False)